# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [6]:
import requests
import pyowm
import json

# Load .env and get API key securely
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OWM_API_KEY")

# Check it worked
print("API Key loaded:", api_key[:5] + "..." if api_key else "NOT FOUND")


API Key loaded: 2a2b0...


## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [20]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 288.34,
  'feels_like': 287.17,
  'temp_min': 287.01,
  'temp_max': 289.36,
  'pressure': 1007,
  'humidity': 48,
  'sea_level': 1007,
  'grnd_level': 1003},
 'visibility': 10000,
 'wind': {'speed': 7.72, 'deg': 70, 'gust': 12.86},
 'clouds': {'all': 92},
 'dt': 1745079366,
 'sys': {'type': 2,
  'id': 268730,
  'country': 'GB',
  'sunrise': 1745038553,
  'sunset': 1745089374},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [8]:
# Import the OWM wrapper and set up the API key
from pyowm.owm import OWM

# Initialize the OWM object with your API key
owm = OWM(api_key)

# Create a weather manager to access weather data
mgr = owm.weather_manager()

# Request current weather for London
observation = mgr.weather_at_place("London,GB")
weather = observation.weather

# Print weather status
print("Weather status:", weather.detailed_status)

Weather status: overcast clouds


## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [11]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 15.43°C, Humidity: 49%, Wind Speed: 7.72 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [12]:
# Import the OWM wrapper and set up the API key
from pyowm.owm import OWM

# Initialize the OWM object
owm = OWM(api_key)

# Create the weather manager
mgr = owm.weather_manager()

# Get the weather observation for London
observation = mgr.weather_at_place("London,GB")
weather = observation.weather

# Extract temperature, humidity and wind speed
temperature = weather.temperature('celsius')['temp']
humidity = weather.humidity
wind_speed = weather.wind()['speed']

# Print the results
print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")


Temperature: 15.43°C, Humidity: 48%, Wind Speed: 9.26 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [13]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [17]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    temp_kelvin = entry['main']['temp']    # original temp in Kelvin
    temp_celsius = temp_kelvin - 273.15    # convert to Celsius
    print(f"Time: {entry['dt_txt']}, Temp: {temp_celsius:.2f}°C")

Time: 2025-04-19 18:00:00, Temp: 23.96°C
Time: 2025-04-19 21:00:00, Temp: 25.15°C
Time: 2025-04-20 00:00:00, Temp: 24.53°C
Time: 2025-04-20 03:00:00, Temp: 21.58°C
Time: 2025-04-20 06:00:00, Temp: 19.50°C
Time: 2025-04-20 09:00:00, Temp: 15.61°C
Time: 2025-04-20 12:00:00, Temp: 13.48°C
Time: 2025-04-20 15:00:00, Temp: 15.01°C
Time: 2025-04-20 18:00:00, Temp: 17.83°C
Time: 2025-04-20 21:00:00, Temp: 18.55°C
Time: 2025-04-21 00:00:00, Temp: 16.47°C
Time: 2025-04-21 03:00:00, Temp: 14.08°C
Time: 2025-04-21 06:00:00, Temp: 12.61°C
Time: 2025-04-21 09:00:00, Temp: 12.09°C
Time: 2025-04-21 12:00:00, Temp: 12.14°C
Time: 2025-04-21 15:00:00, Temp: 11.96°C
Time: 2025-04-21 18:00:00, Temp: 12.57°C
Time: 2025-04-21 21:00:00, Temp: 12.55°C
Time: 2025-04-22 00:00:00, Temp: 11.79°C
Time: 2025-04-22 03:00:00, Temp: 11.41°C
Time: 2025-04-22 06:00:00, Temp: 11.63°C
Time: 2025-04-22 09:00:00, Temp: 11.91°C
Time: 2025-04-22 12:00:00, Temp: 14.07°C
Time: 2025-04-22 15:00:00, Temp: 18.93°C
Time: 2025-04-22

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [15]:
# Get 3-hour forecast for New York, US
forecast = mgr.forecast_at_place("New York,US", "3h").forecast

# Loop through each forecast entry and print time + temperature
for weather in forecast:
    print(f"Time: {weather.reference_time('iso')}, Temp: {weather.temperature('celsius')['temp']}°C")

Time: 2025-04-19 18:00:00+00:00, Temp: 23.96°C
Time: 2025-04-19 21:00:00+00:00, Temp: 25.15°C
Time: 2025-04-20 00:00:00+00:00, Temp: 24.53°C
Time: 2025-04-20 03:00:00+00:00, Temp: 21.58°C
Time: 2025-04-20 06:00:00+00:00, Temp: 19.5°C
Time: 2025-04-20 09:00:00+00:00, Temp: 15.61°C
Time: 2025-04-20 12:00:00+00:00, Temp: 13.48°C
Time: 2025-04-20 15:00:00+00:00, Temp: 15.01°C
Time: 2025-04-20 18:00:00+00:00, Temp: 17.83°C
Time: 2025-04-20 21:00:00+00:00, Temp: 18.55°C
Time: 2025-04-21 00:00:00+00:00, Temp: 16.47°C
Time: 2025-04-21 03:00:00+00:00, Temp: 14.08°C
Time: 2025-04-21 06:00:00+00:00, Temp: 12.61°C
Time: 2025-04-21 09:00:00+00:00, Temp: 12.09°C
Time: 2025-04-21 12:00:00+00:00, Temp: 12.14°C
Time: 2025-04-21 15:00:00+00:00, Temp: 11.96°C
Time: 2025-04-21 18:00:00+00:00, Temp: 12.57°C
Time: 2025-04-21 21:00:00+00:00, Temp: 12.55°C
Time: 2025-04-22 00:00:00+00:00, Temp: 11.79°C
Time: 2025-04-22 03:00:00+00:00, Temp: 11.41°C
Time: 2025-04-22 06:00:00+00:00, Temp: 11.63°C
Time: 2025-04-

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [27]:
params = {
    'appid': api_key,
    'units': 'metric'
}

city = 'London,GB'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url, params=params)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 15.19°C, Fahrenheit: 59.342°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [26]:
# Get the weather observation for London
observation = mgr.weather_at_place("London,GB")
weather = observation.weather

# Get the temperature directly in Celsius and Fahrenheit using the wrapper
temp_celsius = weather.temperature('celsius')['temp']
temp_fahrenheit = weather.temperature('fahrenheit')['temp']

# Show results
print(f"Temperature in Celsius: {temp_celsius:.2f}°C, Fahrenheit: {temp_fahrenheit:.2f}°F")

Temperature in Celsius: 14.86°C, Fahrenheit: 58.75°F
